In [8]:
from BeamFE2 import HermitianBeam_2D as HB
from BeamFE2 import BeamSections as sections
from BeamFE2 import Material
from BeamFE2 import Structure
from BeamFE2 import Node
%matplotlib inline

ModuleNotFoundError: No module named 'BeamFE2'

In [ ]:
# declaring some constants
LENGTH = 400  # length of the cantilever
NR_BEAMS = 3  # number of beam FEs

In [ ]:
# nodes in a straight line
_nodes = [Node.Node.from_dict(adict={'ID': i+1, 'coords': (i*LENGTH/NR_BEAMS, 0)}) for i in range(NR_BEAMS+1)]

In [ ]:
# beams
# first, a rectangle cross section
section_column = sections.Rectangle(height=3, width=10)

# material: steel with nominal stiffness, weight
mat = Material.Steel()

# creating the beams between the nodes
_beams = [HB.HermitianBeam2D.from_dict(adict=
                                       {'ID': i, 
                                        'i': _nodes[i], 'j': _nodes[i+1],
                                        'E': mat.E, 
                                        'rho': mat.rho, 
                                        'I': section_column.I['x'], 
                                        'A': section_column.A,
                                       })
          for i in range(NR_BEAMS)]

In [ ]:
# supports
BCs = {1: ['ux', 'uy', 'rotz']}

In [ ]:
# the structures
structure = Structure.Structure(beams=_beams, supports=BCs)

In [ ]:
# adding the loads - nodal loads
# 
structure.add_nodal_load(nodeID=_nodes[-1].ID, dynam={'FX': -100, 'FY': -200, 'MZ': 0}, clear=True)

In [ ]:
# adding the loads - beam internal loads
structure.add_internal_loads(beam=structure.beams[0], loadtype='uniform axial force', value=1)
structure.add_internal_loads(beam=structure.beams[0], loadtype='concentrated axial force', value=200, position=0.3)
structure.add_internal_loads(beam=structure.beams[0], loadtype='concentrated moment', value=-1000000, position=0.5)
structure.add_internal_loads(beam=structure.beams[0], loadtype='uniform perpendicular force', value=-10)

In [ ]:
# solving it
structure.solver['linear static'].solve()

In [ ]:
structure.draw(analysistype='linear static')